In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [901]:
import praw
import numpy as np
import pandas as pd
from collections import defaultdict
from time import strftime, gmtime
import re
from tqdm import tqdm
import pprint
import os

In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [902]:
# settings.py
from dotenv import load_dotenv
load_dotenv()

# OR, explicitly providing path to '.env'
from pathlib import Path  # python3 only
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

True

True

In [903]:
client_id = os.getenv("r_client_id")
client_secret = os.getenv("r_client_secret")
user_agent = os.getenv("r_user_agent")
username = os.getenv("r_username")
password = os.getenv("r_password")

In [904]:
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent,
                     username=username,
                     password=password)

In [873]:
df_game = pd.read_json('data/Game_1000_85.json')
df_game = df_game.reset_index()
pd.set_option("max_rows", None)
df_game.head()

,index,name,released,parent_platforms,platforms,added,added_by_status,genres,metacritic,slug,stores,tags
0,0,Grand Theft Auto V,2013-09-17,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 4, 'name': 'PC', 'slug': ...",11725,"{'yet': 246, 'owned': 7345, 'beaten': 2902, 't...","[{'id': 4, 'name': 'Action', 'slug': 'action',...",97,grand-theft-auto-v,"[{'id': 290375, 'store': {'id': 3, 'name': 'Pl...","[{'id': 40836, 'name': 'Full controller suppor..."
1,1,Portal 2,2011-04-19,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 16, 'name': 'PlayStation ...",10341,"{'yet': 312, 'owned': 6658, 'beaten': 2850, 't...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...",95,portal-2,"[{'id': 33916, 'store': {'id': 7, 'name': 'Xbo...","[{'id': 40833, 'name': 'Captions available', '..."
2,2,The Witcher 3: Wild Hunt,2015-05-18,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 4, 'name': 'PC', 'slug': ...",9982,"{'yet': 513, 'owned': 5844, 'beaten': 2246, 't...","[{'id': 4, 'name': 'Action', 'slug': 'action',...",93,the-witcher-3-wild-hunt,"[{'id': 354780, 'store': {'id': 5, 'name': 'GO...","[{'id': 40836, 'name': 'Full controller suppor..."
3,3,The Elder Scrolls V: Skyrim,2011-11-11,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 4, 'name': 'PC', 'slug': ...",9252,"{'yet': 240, 'owned': 6019, 'beaten': 1907, 't...","[{'id': 4, 'name': 'Action', 'slug': 'action',...",94,the-elder-scrolls-v-skyrim,"[{'id': 49792, 'store': {'id': 7, 'name': 'Xbo...","[{'id': 97, 'name': 'Action RPG', 'slug': 'act..."
4,4,Tomb Raider (2013),2013-03-05,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 18, 'name': 'PlayStation ...",9201,"{'yet': 263, 'owned': 6362, 'beaten': 2164, 't...","[{'id': 4, 'name': 'Action', 'slug': 'action',...",86,tomb-raider,"[{'id': 79036, 'store': {'id': 4, 'name': 'App...","[{'id': 40836, 'name': 'Full controller suppor..."


In [885]:
def reddit_info(subr):
    print(subr.title)
    pprint.pprint(vars(subr))

In [144]:
def filter_key(key, df=df_game):
    return [True if re.search(key, i, re.IGNORECASE) else False for i in df.name]

In [693]:
df_game[filter_key('total war')]

,index,name,released,parent_platforms,platforms,added,added_by_status,genres,metacritic,slug,...,sub1,sub_title1,sub_subscribers1,sub_description1,sub_pub_desc1,sub2,sub_title2,sub_subscribers2,sub_description2,sub_pub_desc2
78,174,Total War: SHOGUN 2,2011-03-14,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 6, 'name': 'Linux', 'slug...",3757,"{'yet': 134, 'owned': 3396, 'beaten': 99, 'top...","[{'id': 10, 'name': 'Strategy', 'slug': 'strat...",90,total-war-shogun-2,...,gaming,gaming,26333244.0,"**If your submission does not appear, do not d...",A subreddit for (almost) anything related to g...,GameDeals,GameDeals,737131.0,##### [Submit a new deal](/r/GameDeals/submit?...,A community for sharing and discussing game de...
176,723,Medieval II: Total War,2007-11-27,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 6, 'name': 'Linux', 'slug...",1581,"{'yet': 72, 'owned': 1336, 'beaten': 104, 'top...","[{'id': 10, 'name': 'Strategy', 'slug': 'strat...",88,medieval-ii-total-war,...,gaming,gaming,26333245.0,"**If your submission does not appear, do not d...",A subreddit for (almost) anything related to g...,GameDeals,GameDeals,737131.0,##### [Submit a new deal](/r/GameDeals/submit?...,A community for sharing and discussing game de...
181,743,Rome: Total War - Collection,2007-08-28,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 4, 'name': 'PC', 'slug': ...",1550,"{'yet': 75, 'owned': 1352, 'beaten': 74, 'topl...","[{'id': 10, 'name': 'Strategy', 'slug': 'strat...",92,rome-total-war-collection,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,802,Empire: Total War,2009-03-03,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 4, 'name': 'PC', 'slug': ...",1459,"{'yet': 64, 'owned': 1230, 'beaten': 85, 'topl...","[{'id': 10, 'name': 'Strategy', 'slug': 'strat...",90,empire-total-war,...,totalwar,totalwar,235357.0,### [Click here to set your flair!](https://rt...,A subreddit for the Total War strategy game se...,gaming,gaming,26333247.0,"**If your submission does not appear, do not d...",A subreddit for (almost) anything related to g...
212,1040,Total War: WARHAMMER,2016-05-24,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 5, 'name': 'macOS', 'slug...",1216,"{'yet': 65, 'owned': 935, 'beaten': 100, 'topl...","[{'id': 5, 'name': 'RPG', 'slug': 'role-playin...",86,total-war-warhammer,...,totalwar,totalwar,235357.0,### [Click here to set your flair!](https://rt...,A subreddit for the Total War strategy game se...,gaming,gaming,26333244.0,"**If your submission does not appear, do not d...",A subreddit for (almost) anything related to g...


In [564]:
c = re.search(r'\s(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$|\s(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})[\s:-]', 'GTA V: sds').group().strip()
c
#ob1.romanToInt(c)

'V:'

In [598]:
def string_roman2i(text):
    c = re.search(r'\s(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$|\s(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})[\s:-]', text).group()
    c_roman = re.sub(r'[^\w]+', '', c)
    r2i = Solution()
    d = r2i.romanToInt(c_roman)
    c_new = re.sub(r'([\w]+)', str(d), c)
    #print(c, d, c_new)
    text_new = re.sub(r'\s(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$|\s(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})[\s:-]', c_new, text)
    
    return text_new

In [540]:
class Solution(object):
    def romanToInt(self, s):

        roman = {'I':1,'V':5,'X':10,'L':50,'C':100,'D':500,'M':1000,'IV':4,'IX':9,'XL':40,'XC':90,'CD':400,'CM':900}
        i = 0
        num = 0
        while i < len(s):
            if i+1<len(s) and s[i:i+2] in roman:
                num+=roman[s[i:i+2]]
                i+=2
            else:
                #print(i)
                num+=roman[s[i]]
                i+=1
        return num
ob1 = Solution()
print(ob1.romanToInt("III"))
print(ob1.romanToInt("CDXLIII"))

3
443


In [872]:
def sep_title(name):
    '''Function to seperate game title into main title and sub title if there's any, 
    use number, parentheses, colon as mark to seperate titles. Return full, main and sub titles '''
    
    name = re.sub(r'[^\w,\.\'\?!:\+\(\)-]+', ' ', name)
    pattern = re.compile(r'([a-zA-Z\s\'\.-]+[0-9\s\,]*)\s?[:!\(]?\s?([a-zA-Z0-9\s\'-]+)?\W?')
    full = pattern.search(name).group(0)
    main = pattern.search(name).group(1)
    sub = pattern.search(name).group(2)
    
    ''' Set exception for total war as a few entries of this game has reversed entry of main and sub title,
    one example is rome: total war, if use same way to seperate, r/rome will win against r/totalwar due to higher matching
    More exception might be needed when we include more games'''
    
    if re.search('total war', full, re.IGNORECASE):
        main = 'total war'
    
    return full, main, sub

In [874]:
sep_test = pd.DataFrame()
sep_test['name'] = df_game.name
sep_test['slug'] = df_game.slug
for index, row in sep_test.iterrows():
    #print(index)
    full, main, sub = sep_title(row['name'])
    sep_test.loc[index, 'full']=full
    sep_test.loc[index, 'main']=main
    sep_test.loc[index, 'sub']=sub

sep_test.head()

,name,slug,full,main,sub
0,Grand Theft Auto V,grand-theft-auto-v,Grand Theft Auto V,Grand Theft Auto V,NaN
1,Portal 2,portal-2,Portal 2,Portal 2,NaN
2,The Witcher 3: Wild Hunt,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,The Witcher 3,Wild Hunt
3,The Elder Scrolls V: Skyrim,the-elder-scrolls-v-skyrim,The Elder Scrolls V: Skyrim,The Elder Scrolls V,Skyrim
4,Tomb Raider (2013),tomb-raider,Tomb Raider (2013),Tomb Raider,2013


In [864]:
df_sub_result = search_sub(df_game)

232it [07:00,  1.81s/it]


In [909]:
df_sub_result

,name,sub_name,sub_pub_desc,subscribers
0,Grand Theft Auto V,GTAV,"Grand Theft Auto V, also known as GTA 5 or GTA...",256651.0
1,Portal 2,Portal,Welcome to Aperture Laboratories. The subreddi...,77681.0
2,The Witcher 3: Wild Hunt,thewitcher3,For everything Witcher related.,67841.0
3,The Elder Scrolls V: Skyrim,skyrim,,891740.0
4,Tomb Raider (2013),TombRaider,A subreddit for people interested in Lara Crof...,30280.0
5,Left 4 Dead 2,l4d2,Left 4 Dead series related. Please refer to th...,20617.0
6,Borderlands 2,Borderlands2,The Borderlands 2 Reddit.\nPost and discuss an...,150956.0
7,BioShock Infinite,BioshockInfinite,"A place to share all things Bioshock Infinite,...",9673.0
8,Portal,Portal,Welcome to Aperture Laboratories. The subreddi...,77681.0
9,Limbo,NaN,NaN,NaN


Improvement target:
1. Include acronym of game to search for subreddit use only acronym (e.g. KOToR), also important for comment search if decide to include games without subreddit, or search comment to acquire more information, (include Warhammer 40,000 to warhammer 40k) also some known as name which is not acronym, but use part of the name
2. Better way to retract main title and sub-title, remove edition suffix, combined multiple entries of same game and only recomment the best edition or most proper edition 
3. Within subreddit for series of game, retract comment topic to each specific game to analysis more accurate information
4. Better way to handle tv show vs game subreddit, current method promote some random subreddit to put game in their title

In [871]:
search_sub_test(df_game.loc[[212],:], bonus_point=100, penalty_point=100, min_score=0, min_subscribers=0)

1it [00:00,  2.35it/s]

   index             game_name           sub_name  title_score  des_score  \
0      2  Total War: WARHAMMER           totalwar        100.0      100.0   
1      1  Total War: WARHAMMER     totalwarhammer        100.0      100.0   
2      3  Total War: WARHAMMER      TotalWarArena        100.0      100.0   
3      4  Total War: WARHAMMER     EmpireTotalWar        100.0      100.0   
4      5  Total War: WARHAMMER             gaming         43.0       45.0   
5      0  Total War: WARHAMMER  Totalwarwarhammer        100.0        0.0   

  penalty  bonus                                       sub_pub_desc  \
0   False  False  A subreddit for the Total War strategy game se...   
1   False  False  A subreddit dedicated to the Creative Assembly...   
2   False  False  A subreddit for the game Total War Arena made ...   
3   False  False  Things you've done on Empire total war like in...   
4   False  False  A subreddit for (almost) anything related to g...   
5   False  False                  

In [859]:
def search_sub_test(df, bonus_point = 0, penalty_point = 0, min_score = 0, min_subscribers = 0):
    '''Test function for search_sub, instead of store all information into one final dataframe and return it,
    this function print dataframe for each game's search result, ratings and rankings to trouble shoot and tune each parameters
    for score function '''
    
    df_sub = pd.DataFrame()
    df_sub['name'] = df.name
    
    for index, row in tqdm(df.iterrows()):
        #print('{}.. Searching game {}. Time: {}.'.format(index, game, strftime("%H:%M:%S", gmtime())))
        
        # Intial empty set to store potential subreddit for each game
        sub_set = set()
        df_sub_game = pd.DataFrame()
        
        # Prepare search string, full_name, main title, sub title
        full, main_title, sub_title = sep_title(row['name'])
        names = [full, main_title]
        name_list = []
        
        # Remove None from name_list
        for name in names:
            if name != None:
                name_list.append(name)
        
        # Search subreddit from all names
        for key in name_list:
            if key != None:
                game_sub = reddit.subreddits.search(key, limit=4)
                # Store reddit.subreddit to sub_list
                for sub in game_sub:
                    #print(sub.display_name)
                    if sub.over18 == False and sub.subscribers >= min_subscribers:
                        sub_set.add(sub)

        sub_list = list(sub_set)
        #print(sub_list)
        

        # For each subreddit in sub_list, grade matching result use grade_game_sub
        if len(sub_list) > 0:
            for i, entry in enumerate(sub_list):
                title_score, des_score, penalty, bonus = grade_game_sub(name_list, entry)
                subscribers = entry.subscribers
                # Check if title_score or des_score >= minimum_score
                if title_score >= min_score:
                    df_sub_game.loc[i, 'game_name'] = full
                    #df_sub_game.loc[i, 'subreddit'] = entry
                    df_sub_game.loc[i, 'sub_name'] = entry.display_name
                    df_sub_game.loc[i, 'title_score'] = title_score
                    df_sub_game.loc[i, 'des_score'] = des_score
                    df_sub_game.loc[i, 'penalty'] = penalty
                    df_sub_game.loc[i, 'bonus'] = bonus
                    df_sub_game.loc[i, 'sub_pub_desc'] = entry.public_description
                    df_sub_game.loc[i, 'subscribers'] = entry.subscribers
                    score = title_score * 1.7 + des_score * 1.2  + np.log(subscribers)*8 - penalty * penalty_point + bonus * bonus_point
                    df_sub_game.loc[i, 'score'] = score
                    #print(title_score)
                    #print(entry.display_name)
            if len(df_sub_game)>0:    
                df_sub_game = df_sub_game.sort_values(['score'], ascending=False).reset_index()
                
                #df_sub.loc[index, 'subreddit'] = df_sub_game.loc[0, 'subreddit']   
                print(df_sub_game)

    #return df_sub
    

In [861]:
def search_sub(df, bonus_point=100, penalty_point=100, min_score=75, min_subscribers=2000):
    df_sub = pd.DataFrame()
    df_sub['name'] = df.name
    
    for index, row in tqdm(df.iterrows()):
        #print('{}.. Searching game {}. Time: {}.'.format(index, game, strftime("%H:%M:%S", gmtime())))
        
        # Intial empty set to store potential subreddit for each game
        sub_set = set()
        df_sub_game = pd.DataFrame()
        
        # Prepare search string, full_name, main title, sub title
        full, main_title, sub_title = sep_title(row['name'])
        names = [full, main_title]
        name_list = []
        
        # Remove None from name_list
        for name in names:
            if name != None:
                name_list.append(name)
                
        # Search subreddit from all names
        for key in name_list:
            if key != None:
                game_sub = reddit.subreddits.search(key, limit=4)
                # Store reddit.subreddit to sub_list
                for sub in game_sub:
                    #print(sub.display_name)
                    if sub.over18 == False and sub.subscribers >= min_subscribers:
                        sub_set.add(sub)
        
        sub_list = list(sub_set)

        # For each subreddit in sub_list, grade matching result use grade_game_sub
        if len(sub_list) > 0:
            for i, entry in enumerate(sub_list):
                title_score, des_score, penalty, bonus = grade_game_sub(name_list, entry)
                subscribers = entry.subscribers
                
                
                # Check if title_score or des_score >= minimum_score
                if title_score >= min_score:
                    df_sub_game.loc[i, 'sub_name'] = entry.display_name
                    df_sub_game.loc[i, 'sub_pub_desc'] = entry.public_description
                    df_sub_game.loc[i, 'subscribers'] = entry.subscribers
                    score = title_score * 1.7 + des_score * 1.2  + np.log(subscribers)*8 - penalty * penalty_point + bonus * bonus_point
                    df_sub_game.loc[i, 'score'] = score
                
            if len(df_sub_game)>0:    
                df_sub_game = df_sub_game.sort_values('score', ascending=False).reset_index()
                df_sub.loc[index, 'sub_name'] = df_sub_game.loc[0, 'sub_name']
                df_sub.loc[index, 'sub_pub_desc'] = df_sub_game.loc[0, 'sub_pub_desc']
                df_sub.loc[index, 'subscribers'] = df_sub_game.loc[0, 'subscribers']

    return df_sub
    

In [767]:
def grade_game_sub(name, sub):    
    name_list = []
    roman_compiler = re.compile(r'\s(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$|\s(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})[\s:-]')
    for i in name:
        name_list.append(i)
        #name_list.append(i.replace(' ', ''))
        if roman_compiler.search(i):
            name_list.append(string_roman2i(i))
    name_list = sep_num([n.lower() for n in name_list])
    pub_des = sub.public_description
    des = sub.description
    sub_name = sub.display_name
    sub_title = sub.title
    str_list = [pub_des, des, sub_name, sub_title]
    search_str = []
    for entry in str_list:
        if entry != None:
            search_str.append(entry.lower())
    search_str = sep_num(search_str)
    for i in range(len(search_str)):
        search_str[i] = re.sub("[^0-9a-zA-Z\s\'\.,-]+", "", search_str[i])
    search_str_des = search_str[:2]
    search_str_title = search_str[2:]
    
    #print(search_str)
    #print(name_list)
    des_score = 0
    title_score = 0
    # Define penalty list contain tv show, movies etc. that could confuse with video game
    penalty_list = ['tv show', 'movies', 'movie', 'tv shows', 'tv-series', 'tv series']
    penalty_status = False
    
    bonus_status = False
    
    # Grade matching score based on description
    for entry in search_str_des:
        #print('Name:', name)
        #print('entry:',entry)
        #print([fuzz.partial_ratio(n, entry) for n in name_list])
        s = np.max([fuzz.partial_ratio(n, entry) for n in name_list])
        if np.max([fuzz.partial_ratio(n, entry) for n in penalty_list]) == 100:
            penalty_status = True
            

        #print('Score:', [fuzz.partial_ratio(n, entry) for n in name])
        #print('\n')
        if s > des_score:
            des_score = s
    
    if re.search('game', sub_name, re.IGNORECASE):
        bonus_status = True
    # Grade matching score based on subreddit title
    s_name = np.max([fuzz.ratio(n, search_str_title[0]) for n in name_list])
    s_title = np.max([fuzz.partial_ratio(n, search_str_title[1]) for n in name_list])
    
    '''for entry in search_str_title:
        entry = re.sub("[^0-9a-zA-Z\s\'\.,-]+", "", entry)
        #print('Name:', name)
        #print('entry:',entry)
        #print([fuzz.partial_ratio(n, entry) for n in name_list])
        s = np.max([fuzz.ratio(n, entry) for n in name_list])
        #print('Score:', [fuzz.partial_ratio(n, entry) for n in name])
        #print('\n')
        if s > title_score:
            title_score = s '''

    title_score = np.max([s_name, s_title])
    
    return title_score, des_score, penalty_status, bonus_status

In [8]:
def filter_key(df, key):
    return np.logical_and([True if re.search(key, i, re.IGNORECASE) else False for i in df.name],
                        [True if re.search(key, str(i), re.IGNORECASE) else False for i in df.description])

In [14]:
def sep_num(strings):
    return [re.sub(r"(-?[0-9]+\.?[0-9]*)", r" \1 ", string).strip() for string in strings]

In [ ]:
def filter_subreddit(score, df=df_game):
    df_sub = pd.DataFrame()
    df_sub['name'] = df.name
    df_sub['']
    #df_sub['sub0'] = None
    #df_sub['sub1'] = None
    #df_sub['sub2'] = None
    for index, row in df.iterrows():
        sub_score = np.empty(3)
        for i in range(3):
            sub_score[i] = row['fpr'+str(i)]
        max_score = sub_score.max()
        max_sub=[]
        if max_score >= score:
            for i in range(3):
                if sub_score[i] == max_score:
                    max_sub.append(i)
            for ind, n in enumerate(max_sub):
                df_sub.loc[index, 'sub'+str(ind)] = row['sub_name'+str(n)]
                df_sub.loc[index, 'score'+str(ind)] = row['fpr'+str(n)]
    return df_sub
                

In [908]:
df_game.head()

,index,name,released,parent_platforms,platforms,added,added_by_status,genres,metacritic,slug,stores,tags
0,0,Grand Theft Auto V,2013-09-17,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 4, 'name': 'PC', 'slug': ...",11725,"{'yet': 246, 'owned': 7345, 'beaten': 2902, 't...","[{'id': 4, 'name': 'Action', 'slug': 'action',...",97,grand-theft-auto-v,"[{'id': 290375, 'store': {'id': 3, 'name': 'Pl...","[{'id': 40836, 'name': 'Full controller suppor..."
1,1,Portal 2,2011-04-19,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 16, 'name': 'PlayStation ...",10341,"{'yet': 312, 'owned': 6658, 'beaten': 2850, 't...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...",95,portal-2,"[{'id': 33916, 'store': {'id': 7, 'name': 'Xbo...","[{'id': 40833, 'name': 'Captions available', '..."
2,2,The Witcher 3: Wild Hunt,2015-05-18,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 4, 'name': 'PC', 'slug': ...",9982,"{'yet': 513, 'owned': 5844, 'beaten': 2246, 't...","[{'id': 4, 'name': 'Action', 'slug': 'action',...",93,the-witcher-3-wild-hunt,"[{'id': 354780, 'store': {'id': 5, 'name': 'GO...","[{'id': 40836, 'name': 'Full controller suppor..."
3,3,The Elder Scrolls V: Skyrim,2011-11-11,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 4, 'name': 'PC', 'slug': ...",9252,"{'yet': 240, 'owned': 6019, 'beaten': 1907, 't...","[{'id': 4, 'name': 'Action', 'slug': 'action',...",94,the-elder-scrolls-v-skyrim,"[{'id': 49792, 'store': {'id': 7, 'name': 'Xbo...","[{'id': 97, 'name': 'Action RPG', 'slug': 'act..."
4,4,Tomb Raider (2013),2013-03-05,"[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'platform': {'id': 18, 'name': 'PlayStation ...",9201,"{'yet': 263, 'owned': 6362, 'beaten': 2164, 't...","[{'id': 4, 'name': 'Action', 'slug': 'action',...",86,tomb-raider,"[{'id': 79036, 'store': {'id': 4, 'name': 'App...","[{'id': 40836, 'name': 'Full controller suppor..."


In [878]:
df_sub_result.to_csv('data/df_sub.csv')

In [889]:
sub_list = df_sub_result.sub_name.unique()
sub_list

array(['GTAV', 'Portal', 'thewitcher3', 'skyrim', 'TombRaider', 'l4d2',
       'Borderlands2', 'BioshockInfinite', nan, 'HalfLife3', 'Bioshock',
       'tf2', 'reddeadredemption', 'GTA', 'DotA2', 'TheWalkingDeadGame',
       'GodofWar', 'RocketLeague', 'metalgearsolid', 'horizon',
       'uncharted', 'civ', 'thelastofus', 'masseffect', 'SpidermanPS4',
       'farcry', 'HalfLife', 'DeadSpace', 'darksouls3', 'Xcom',
       'bloodborne', 'pathofexile', 'GlobalOffensive', 'dishonored',
       'Fallout', 'Psychonaut', 'batman', 'JourneyPS3', 'Fez',
       'HollowKnight', 'SaintsRow', 'StardewValley', 'stanleyparable',
       'assassinscreed', 'modernwarfare', 'Deusex', 'nier',
       'residentevil', 'MonsterHunterWorld', 'Rayman', 'rally',
       'dragonage', 'Undertale', 'Torchlight', 'totalwar', 'XCOM2',
       'battlefield_one', 'Wolfenstein', 'CompanyOfHeroes',
       'Titanfall_2_', 'OriAndTheBlindForest', 'Cuphead', 'Sekiro',
       'TheWitness', 'apexlegends', 'JetSetRadio', 'Bayonet

In [899]:
a=0
user = set()
for i in range(3):
    for comment in reddit.subreddit(sub_list[0]).comments(limit=None):
        user.add(comment.author.name)
        a += 1
a

2982

In [900]:
len(user)

620

In [907]:
len(redditor)

619

In [906]:
df_sub_result.groupby('sub_name').name.count().sort_values(ascending=False)

sub_name
totalwar                5
uncharted               4
batman                  4
CallOfDuty              4
GTA                     4
civ                     4
Fallout                 3
ForzaHorizon            3
assassinscreed          3
HalfLife3               3
thewitcher3             3
DivinityOriginalSin     3
Deusex                  3
residentevil            3
farcry                  2
dishonored              2
metalgearsolid          2
darksouls3              2
ElderScrolls            2
l4d2                    2
GlobalOffensive         2
masseffect              2
Doom                    2
DeadSpace               2
CompanyOfHeroes         2
OriAndTheBlindForest    2
reddeadredemption       2
skyrim                  2
Bioshock                2
Portal                  2
Rayman                  2
thelastofus             2
dragonage               2
IntoTheBreach           1
JourneyPS3              1
Injustice2MobileGame    1
JetSetRadio             1
zelda                   1
Ker